<a href="https://colab.research.google.com/github/kobi3028/AttacksonImplementationsCourseBook/blob/master/Labs/ROB_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micro architecture buffers sizes testing tool
This is a python testing tool for micro architecture buffers sizes like [ROB](https://https://en.wikipedia.org/wiki/Re-order_buffer) size described by [Henry Wong on his blog](https://blog.stuffedcow.net/2013/05/measuring-rob-capacity) inspired by [robsize](https://travisdowns/robsize) github repository.

## Filter number 0, 2 and 3
This uses a series of general-purpose register additions/copy/compare, each one of which should consume a physical register, to ***test the size of the speculative register file***.

## Filter number 1 and 4
These tests use single byte or double byte NOPs to ***detect the Re-ordering buffer (ROB) size***. They should both give the same answer, since only difference is that single byte NOPs tend to disable the use of micro-op cache since they are too dense, while double-byte NOPs use the cache. I wouldn't expect that to affect the apparent ROB size, but it's nice to be able to check!

## Filter number 32 and 33
These ***determine the load buffer size (test 32) and store buffer size (test 33)*** respectively, by using loads and stores as the filler instructions.



## Script parameters
FILTER_NUM   - change filter number to change micro architecture test (use diffrent op code as a filter instructions).

START_ICOUNT - the test will start from START_ICOUNT size of filter opcode block.

END_ICOUNT   - the test will finish at END_ICOUNT size of filter opcode block

LFENCE_MOD   - use lfence opcode insted of close filter instructions block.

TEST_ITS     - number of test iteration (the result is the min max and avg of all iteration).

ASM_ITS      - number of iteration inside the assembly.

In [1]:
# Filter number
FILTER_NUM = 0
# number of filter instruction - start
START_ICOUNT = 4
# number of filter instruction - end
END_ICOUNT = 256
# use lfence insted of filter instractions
LFENCE_MOD = False
# number of test iteration
TEST_ITS = 64
# number of iteration inside the assembly 
ASM_ITS = 0x2000

In [2]:
#!cat /proc/cpuinfo
import mmap
import ctypes
import struct

!pip install -q bokeh
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
from bokeh.models.tickers import FixedTicker
from bokeh.models.formatters import PrintfTickFormatter
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

import numpy as np
np.set_printoptions(formatter={'int':hex})

!pip install -q keystone-engine
from keystone import *

ks = Ks(KS_ARCH_X86, KS_MODE_64)

# maximum number of filler instructions supported 
MAX_ICOUNT = 400;
# we repeat the load/payload pattern "unroll" times
unroll = 17
# stack space created for some tests which read/write the stack
STACK_SPACE = MAX_ICOUNT * unroll * 2 + 100 # 100 arbitrary magic number

In [3]:
# rdtsc func 
rdtsc_asm = '''
rdtsc
shl rdx, 32
or  rax, rdx
ret
'''
byte_code, count = ks.asm(rdtsc_asm)
assert count == 5

rdtsc_buf = mmap.mmap(-1, mmap.PAGESIZE, prot=mmap.PROT_READ | mmap.PROT_WRITE | mmap.PROT_EXEC)
rdtsc_buf.write(bytes(byte_code))

fpointer = ctypes.c_void_p.from_buffer(rdtsc_buf)
func_type = ctypes.CFUNCTYPE(ctypes.c_uint64)
rdtsc = ctypes.cast(ctypes.addressof(fpointer), func_type)

In [4]:
#init_dbuf
dbuf_size = mmap.PAGESIZE*0x10000
size = dbuf_size//8
cycle_length = 0x2000//8
#dbuf = mmap.mmap(-1, dbuf_size, prot=mmap.PROT_READ | mmap.PROT_WRITE | mmap.PROT_EXEC)
dbuf = np.arange(start=0, stop=size, dtype=np.uint64)
#print(dbuf)
np.random.shuffle(dbuf)
#print(dbuf)

dpointer = ctypes.c_void_p.from_buffer(dbuf)
dpointer_address = ctypes.addressof(dpointer)

counter = 0
while counter < size:
  val = dpointer_address + (int(dbuf[counter]) * 8)
  dbuf[counter] = val
  counter += 1

print('data buf address:', hex(dpointer_address))

data buf address: 0x55c74d734000


In [5]:
#alloc read write execute memory
fbuf_size = mmap.PAGESIZE*0x100
fbuf = mmap.mmap(-1, fbuf_size, prot=mmap.PROT_READ | mmap.PROT_WRITE | mmap.PROT_EXEC)

fpointer = ctypes.c_void_p.from_buffer(fbuf)
fpointer_address = ctypes.addressof(fpointer)
print('machine code buf address:', hex(fpointer_address))

machine code buf address: 0x7fba856de000


In [6]:
# filter opcode dict
op_dict = {
    1 : bytes(ks.asm( 'nop')[0]),
    4 : bytes(ks.asm( 'xchg ax, ax')[0]),
    32: bytes(ks.asm( 'mov  ebx, [rsp]')[0]),
    33: bytes(ks.asm( 'mov  [rsp-0x8], ebx')[0]),
    80: bytes(ks.asm( 'add  ebx,ebx')[0]), # for filter_num 0
    81: bytes(ks.asm( 'add  ebp,ebp')[0]), # for filter_num 0
    82: bytes(ks.asm( 'add  esi,esi')[0]), # for filter_num 0
    83: bytes(ks.asm( 'add  edi,edi')[0]), # for filter_num 0
    90: bytes(ks.asm( 'mov  ebx,ebx')[0]), # for filter_num 2
    91: bytes(ks.asm( 'mov  ebp,ebp')[0]), # for filter_num 2
    92: bytes(ks.asm( 'mov  esi,esi')[0]), # for filter_num 2
    93: bytes(ks.asm( 'mov  edi,edi')[0]), # for filter_num 2
   100: bytes(ks.asm( 'cmp  ebx,ebx')[0]), # for filter_num 3
   101: bytes(ks.asm( 'cmp  ebp,ebp')[0]), # for filter_num 3
   102: bytes(ks.asm( 'cmp  esi,esi')[0]), # for filter_num 3
   103: bytes(ks.asm( 'cmp  edi,edi')[0]), # for filter_num 3
    }
def filter_op(filter_num, i):
  if filter_num in [1, 4, 32, 33]:
    return op_dict[filter_num]
  elif filter_num == 0:
    return op_dict[80 + (i%4)]
  elif filter_num == 2:
    return op_dict[90 + (i%4)]
  elif filter_num == 3:
    return op_dict[100 + (i%4)]

  

In [7]:
def make_routine(icount, filter_num, lfence_mode):
  assert icount < MAX_ICOUNT
  assert filter_num in [0, 1, 2, 3, 4, 32, 33]

  p1 = dpointer_address 
  p2 = dpointer_address + (0x801000 // 8) 

  #assemble intel assembly to intel x86_64 CPU byte code
  code = b''
  code += bytes(ks.asm( 'xchg   ax,ax')[0])*8
  code += bytes(ks.asm( 'push   rbx')[0])
  code += bytes(ks.asm( 'push   rbp')[0])
  code += bytes(ks.asm( 'push   rsi')[0])
  code += bytes(ks.asm( 'push   rdi')[0])
  code += bytes(ks.asm( 'push   r8')[0])
  code += bytes(ks.asm( 'push   r9')[0])
  code += bytes(ks.asm(f'sub    rsp,{STACK_SPACE}')[0])
  code += bytes(ks.asm( 'xor    r8d, r8d')[0])
  code += bytes(ks.asm( 'lea    r9,[rsp]')[0])
  code += bytes(ks.asm(f'movabs rcx,{p1}')[0])
  code += bytes(ks.asm(f'movabs rdx,{p2}')[0])
  code += bytes(ks.asm(f'movabs rax,{ASM_ITS}')[0]) #counter

  code += bytes(ks.asm( 'sub    rbx,0x0')[0])
  code += bytes(ks.asm( 'sub    rbp,0x0')[0])
  code += bytes(ks.asm( 'sub    rsi,0x0')[0])
  code += bytes(ks.asm( 'sub    rdi,0x0')[0])
  code += bytes(ks.asm( 'sub    r8, 0x0')[0])
  code += bytes(ks.asm( 'sub    r9, 0x0')[0])
  code += bytes(ks.asm( 'sub    r10,0x0')[0])
  code += bytes(ks.asm( 'sub    r11,0x0')[0])
  code += bytes(ks.asm( 'sub    r12,0x0')[0])
  code += bytes(ks.asm( 'sub    r13,0x0')[0])
  code += bytes(ks.asm( 'sub    r14,0x0')[0])
  code += bytes(ks.asm( 'sub    r15,0x0')[0])
  #padd
  code += bytes(ks.asm( 'nop')[0])*(len(code)%0x10)

  loop_start = len(code)

  u = unroll-1
  k = 0
  while u >= 0:
    code += bytes(ks.asm( 'mov    rcx,QWORD PTR [rcx]')[0])
    for j in range(icount):
      code += filter_op(filter_num, j)
    code += bytes(ks.asm( 'mov    rdx,QWORD PTR [rdx]')[0])
    if lfence_mode:
      code += bytes(ks.asm( 'lfence')[0])
    else:
      for j in range(icount):
        code += filter_op(filter_num, j)
    u -= 1

  #print(bytes(ks.asm( 'mov    rcx,QWORD PTR [rcx]')[0]), bytes(ks.asm( 'mov    rdx,QWORD PTR [rdx]')[0]))
  code += bytes(ks.asm( 'sub   eax, 0x1')[0]) #dec counter
  #print(loop_start - len(code) - 4, bytes(ks.asm(f'jne   {loop_start - len(code) - 4}')[0]))
  code += bytes(ks.asm(f'jne   {loop_start - len(code) - 4}')[0]) #loop if eax != 0 


  code += bytes(ks.asm( 'xchg  ax, ax')[0])*8

  code += bytes(ks.asm(f'add   rsp,{STACK_SPACE}')[0])

  code += bytes(ks.asm( 'pop   r9')[0])
  code += bytes(ks.asm( 'pop   r8')[0])
  code += bytes(ks.asm( 'pop   rdi')[0])
  code += bytes(ks.asm( 'pop   rsi')[0])
  code += bytes(ks.asm( 'pop   rbp')[0])
  code += bytes(ks.asm( 'pop   rbx')[0])

  code += bytes(ks.asm( 'emms')[0])
  code += bytes(ks.asm( 'ret')[0])
  code += bytes(ks.asm( 'nop')[0])*(mmap.PAGESIZE - (len(code) % mmap.PAGESIZE)) #pad byte code with nops
  return code

In [8]:
res = {}

for icount in range(START_ICOUNT, END_ICOUNT):
  routine_code = make_routine(icount, FILTER_NUM, LFENCE_MOD)
  fbuf.write(routine_code)

  #convert byte code to python function 
  routine_pointer = ctypes.c_void_p.from_buffer(fbuf)
  routine_type = ctypes.CFUNCTYPE(ctypes.c_int)
  routine = ctypes.cast(ctypes.addressof(routine_pointer), routine_type)
  routine()

  min_diff = 0x7fffffffffffffff;
  max_diff = 0x0;
  sum_diff = 0x0;

  for j in range(TEST_ITS):
    routine_skip = ctypes.cast(ctypes.addressof(fpointer)+(j%13), routine_type)
    start = rdtsc()
    #run the code
    routine_skip()
    test_ticks = rdtsc() - start

    sum_diff += test_ticks
    if min_diff > test_ticks:
      min_diff = test_ticks

    if max_diff < test_ticks:
      max_diff = test_ticks

  res[icount] = (0.5*min_diff/ASM_ITS/unroll, 0.5*sum_diff/ASM_ITS/unroll/TEST_ITS, 0.5*max_diff/ASM_ITS/unroll)
  #print(f'{icount}, {0.5*min_diff/ASM_ITS/unroll}, {0.5*sum_diff/ASM_ITS/unroll/TEST_ITS}, {0.5*max_diff/ASM_ITS/unroll}')
  fbuf.seek(0)

In [9]:
#Plot the results

title_dict = {
    0 : 'speculative register file size test- (add GP registers)',
    1 : 're-ordering buffer (ROB) size test- (one byte NOP instruction)',
    2 : 'speculative register file size test- (copy GP registers)',
    3 : 'speculative register file size test- (compare GP registers)', 
    4 : 're-ordering buffer (ROB) size test- (two bytes NOP instruction)',
    32: 'load buffer size test', 
    33: 'store buffer size test',
}


p = figure(title=title_dict[FILTER_NUM], 
           y_axis_label='CPU tickes', 
           x_axis_label='iCount')

keys = [key for key in res.keys()]
p.xaxis.ticker = FixedTicker(ticks=keys[::8])
p.xaxis.major_label_orientation = "vertical"
p.xaxis.formatter=PrintfTickFormatter(format="0x%X")

keys = sorted(keys)
min_lst = [res[k][0] for k in keys]
avg_lst = [res[k][1] for k in keys]
max_lst = [res[k][2] for k in keys]


p.line(keys, min_lst, line_width=2, line_color='orange', legend_label='min')
p.line(keys, avg_lst, line_width=2, line_color='blue', legend_label='avg')
p.line(keys, max_lst, line_width=2, line_color='green', legend_label='max')

show(p)